### Strategy (solve_wordle)
- Initial Conditions:
  - Set puzzle_word
  - X = Start with universe of 5 letter words along with freqency of usage.
  - Set current_universe = X
- Start
  - Pick guess (pick the most frequent word in current_universe that we haven't picked previously)
  - If guess == puzzle_word)
    - Goto End
  - Get wordle info about how close guess is to the correct word:
    - wordle_info = create_wordle_info(<guess>, <puzzle_word>)
      - Example wordle_info, create_wordle_info("exact", "crane") = ( [('a', 3)], Dict('x' => (0, 0), 'c' => (1, 1), 'e' => (1, 1), 't' => (0, 0)) )
  - Use this match info to filter existing universe.
    - current_universe = filter_universe(wordle_info, current_universe)
  - Goto Start
- End
    - Return guess 

### Environment
- We assume that the server for this notebook has been launched at the top level for this project
  as below we look for the Wordle database in the directory:  ENV["PWD"] * "/data/" .
- The Wordle database is a CSV file with schema: word, freq. Here.
    - The values in the word column are strings assumed to match: r"[a-z]{N}", where N is a integer -- N is the same value for all rows.
    - The items in the freq column are strings that represent floating point numbers. 

### Use/Import Packages

In [1]:
using DataFrames
using StatsBase
using Statistics
using CSV
import Wordle
using Wordle: create_wordle_info, filter_universe, freq_letter_strat, solve_wordle

### Functions to Solve Wordle Puzzle

### Read in Wordle Words
- Five letter words and their frequencies are stored in two files.
- Read them in and create a DataFrame, then sort the words from most to least used.
- Words and frequencies obtained from the Mathematica function WordFrequencyData.

### Setup Wordle Database

In [2]:
## Change this path to point to the wordle_db.csv path.
## Load the Wordle database as a Dataframe with schema: word(5 letter lowercase word as String),freq(Relative frequency of word as Float64)
const WORDLE_DF =  DataFrame(CSV.File(joinpath(dirname(pathof(Wordle)), "../data",  "wordle_db.csv"); header=6, types=[String7, Float64], comment="#"));

### Testing

In [3]:
names(WORDLE_DF)

2-element Vector{String}:
 "word"
 "freq"

In [4]:
pathof(Wordle)

"/home/rsm/proj/github/julia/pkgs/Wordle/src/Wordle.jl"

In [5]:
create_wordle_info("exact", "crane")

([('a', 3), ('e', -1), ('c', -4)], Dict('x' => (0, 0), 'c' => (1, 1), 'e' => (1, 1), 't' => (0, 0)))

In [6]:
(info, d) = create_wordle_info("there", "their")
println(info)
println(d)

[('t', 1), ('h', 2), ('e', 3), ('r', -4)]
Dict('r' => (1, 1), 'e' => (0, 0))


In [7]:
filter_universe((info, d), WORDLE_DF[!, :word])

1-element Vector{String7}:
 "their"

In [8]:
uni = filter(:word => x -> x in ["where", "state", "other", "child", "there", "their", "about"], WORDLE_DF)
solve_wordle("other", uni)

(Any[("trace", [('t', -1), ('r', -2), ('e', -5)], 7), ("their", [('r', 5), ('t', -1), ('h', -2), ('e', -3)], 3)], 3, :SUCCESS)

In [9]:
create_wordle_info("other", "child")

([('h', -3)], Dict('h' => (1, 1), 't' => (0, 0), 'e' => (0, 0), 'o' => (0, 0), 'r' => (0, 0)))

In [10]:
solve_wordle("child", uni; chk_inputs=true)

(Any[("trace", [('c', -4)], 7)], 2, :SUCCESS)

In [11]:
solve_wordle("otter")

(Any[("trace", [('t', -1), ('r', -2), ('e', -5)], 3585), ("their", [('r', 5), ('t', -1), ('e', -3)], 83), ("enter", [('t', 3), ('e', 4), ('r', 5)], 18), ("outer", [('o', 1), ('t', 3), ('e', 4), ('r', 5)], 6)], 5, :SUCCESS)

In [12]:
solve_wordle("gamma")

(Any[("trace", [('a', -3)], 3585), ("among", [('a', -1), ('m', -2), ('g', -5)], 232), ("gamma", [('g', 1), ('a', 2), ('m', 3), ('m', 4), ('a', 5)], 5)], 3, :SUCCESS)

In [13]:
solve_wordle("windy")

(Any[("trace", Tuple{Char, Int64}[], 3585), ("found", [('n', -4), ('d', -5)], 380), ("dying", [('d', -1), ('y', -2), ('i', -3), ('n', -4)], 6), ("windy", [('w', 1), ('i', 2), ('n', 3), ('d', 4), ('y', 5)], 2)], 4, :SUCCESS)

In [14]:
solve_wordle("donut"; chk_inputs=true)

(Any[("trace", [('t', -1)], 3585), ("might", [('t', 5)], 150), ("doubt", [('d', 1), ('o', 2), ('t', 5), ('u', -3)], 12)], 4, :SUCCESS)

In [15]:
solve_wordle("child")

(Any[("trace", [('c', -4)], 3585), ("child", [('c', 1), ('h', 2), ('i', 3), ('l', 4), ('d', 5)], 81)], 2, :SUCCESS)

In [16]:
solve_wordle("hobby")

(Any[("trace", Tuple{Char, Int64}[], 3585), ("found", [('o', 2)], 380), ("bosom", [('o', 2), ('b', -1)], 33), ("lobby", [('o', 2), ('b', 3), ('b', 4), ('y', 5)], 2)], 5, :SUCCESS)

In [17]:
solve_wordle("wrath")

(Any[("trace", [('r', 2), ('a', 3), ('t', -1)], 3585), ("grant", [('r', 2), ('a', 3), ('t', -5)], 5)], 3, :SUCCESS)

In [18]:
solve_wordle("piano")

(Any[("trace", [('a', 3)], 3585), ("small", [('a', 3)], 67), ("again", [('a', 3), ('i', -4), ('n', -5)], 10), ("piano", [('p', 1), ('i', 2), ('a', 3), ('n', 4), ('o', 5)], 2)], 4, :SUCCESS)

In [19]:
solve_wordle("pixel")

(Any[("trace", [('e', -5)], 3585), ("being", [('e', -2), ('i', -3)], 313), ("field", [('i', 2), ('e', -3), ('l', -4)], 50), ("piles", [('p', 1), ('i', 2), ('e', 4), ('l', -3)], 3)], 5, :SUCCESS)

In [20]:
solve_wordle("often")

(Any[("trace", [('t', -1), ('e', -5)], 3585), ("often", [('o', 1), ('f', 2), ('t', 3), ('e', 4), ('n', 5)], 147)], 2, :SUCCESS)

In [21]:
solve_wordle("sense")

(Any[("trace", [('e', 5)], 3585), ("while", [('e', 5)], 137), ("sense", [('s', 1), ('e', 2), ('n', 3), ('s', 4), ('e', 5)], 39)], 3, :SUCCESS)

#### Run Solver over all Puzzle Words

In [22]:
words = WORDLE_DF[!, :word]
freqs = WORDLE_DF[!, :freq]

guess_len :: Vector{Int64}   = []
freq_val  :: Vector{Float64} = []
successes :: Vector{Symbol}  = []

i = 0
@time begin
    for word in words
        i += 1
              
        ## Use the default strategy -- pick most frequently used word in filtered Wordle universe.
        res = solve_wordle(word; chk_inputs=true)
        ## res = solve_wordle(word; chk_inputs=false, guess_strategy=freq_letter_strat) ## Don't check input contract and use a strategy to pick next guess.

        ## Check for success or failure.
        if res[3] === :SUCCESS
            push!(guess_len, res[2]  )
            push!(freq_val , freqs[i]) 
            push!(successes, res[3]  )
        else
            println("Failure for puzzle word = $word")
            print(res)
            println("\n")
        end
    end
end

  6.949119 seconds (187.02 M allocations: 9.411 GiB, 10.11% gc time, 1.77% compilation time)


#### Gather Statistics for Solver

In [23]:
println("Mean guesses          = $(round(mean(guess_len), digits=2))")
println("Weighted mean quesses = $(round(mean(guess_len, weights(freq_val)), digits=2))")

Mean guesses          = 4.31
Weighted mean quesses = 2.81


In [24]:
solve_wordle("folly")

(Any[("trace", Tuple{Char, Int64}[], 3585), ("found", [('f', 1), ('o', 2)], 380), ("folks", [('f', 1), ('o', 2), ('l', 3)], 4), ("folly", [('f', 1), ('o', 2), ('l', 3), ('l', 4), ('y', 5)], 2)], 4, :SUCCESS)

In [25]:
solve_wordle("cower")

(Any[("trace", [('r', -2), ('c', -4), ('e', -5)], 3585), ("force", [('o', 2), ('r', -3), ('c', -4), ('e', -5)], 51), ("cover", [('c', 1), ('o', 2), ('e', 4), ('r', 5)], 7), ("comer", [('c', 1), ('o', 2), ('e', 4), ('r', 5)], 5), ("coder", [('c', 1), ('o', 2), ('e', 4), ('r', 5)], 4), ("cower", [('c', 1), ('o', 2), ('w', 3), ('e', 4), ('r', 5)], 3)], 6, :SUCCESS)